# Getting started with computer vision by Mnist dataset
___

## Introduction

Computer vision is a field of computer science that deals with the automatic processing of digital images and videos. It has a wide range of applications, including image classification, object detection, and image segmentation.

The MNIST dataset is a popular dataset for computer vision beginners. It is a dataset of handwritten digits, where each row in the dataset corresponds to a single image of size 28x28 pixels.

### Getting Data Ready Part 1

The first step in any computer vision project is to get the data ready. This involves loading the data into a format that can be processed by the computer vision model.

In [ ]:
import torch
import torchvision.transforms as T
import pandas as pd

In [ ]:
torch.cuda.set_device(0)
train_data = pd.read_csv("train.csv")
train_data = pd.concat([train_data,pd.get_dummies(train_data['label'])],axis=1)
del train_data['label']
valid_data = train_data.sample(n=int(len(train_data) * 0.2))
train_data = train_data.drop(valid_data.index)
valid_data = valid_data.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)
test_data = pd.read_csv("test.csv")

### Data Generator Function

In the following code cell, we have created a simple function called data_gen() which receives data from the MNIST dataset and returns a batch of training data. The batch of training data consists of a tuple of two tensors: labels and images. The labels tensor is a tensor of type torch.float32 containing the digit labels for the images in the batch. The images tensor is a tensor of type torch.float32 containing the pixel values for the images in the batch.

In [ ]:

# Define a data generator function
def data_gen(x,train=True):
    """Generates a batch of  data."""
    images = torch.tensor([[x[f'pixel{28 * i + j}'] for j in range(28)] for i in range(28)], dtype=torch.float64).cuda()
    if train :
        labels = torch.tensor(x[-10:], dtype=torch.long).cuda()
        return images.cpu(),labels.cpu()
    return images.cpu()

###  Taking a Sneak Peak at the Data

Let's take a sneak peak at the data structure that we generated from data_gen().



In [ ]:
transform =  T.ToPILImage()
print(data_gen(train_data.loc[6])[1])
# Convert the PyTorch tensor to a Pillow image
(transform(data_gen(train_data.loc[6])[0]/255))

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])


This output shows that the labels tensor is a tensor of type torch.float64 containing the digit label for the image. The images tensor is a tensor of type torch.float64 containing the pixel values for the image.

In [ ]:

def proc_data_saver(df,train=True):
    if train:
      images = [torch.tensor(data_gen(df.loc[row])[0] / 255, dtype=torch.float64) for row in range(df.shape[0])]
      labels = [torch.tensor(data_gen(df.loc[row])[1], dtype=torch.float64) for row in range(df.shape[0])]

      return pd.DataFrame({"Images":images,"labels":labels})
    else:
      images = [torch.tensor(data_gen(df.loc[row],False)/ 255, dtype=torch.float64) for row in range(df.shape[0])]
      return pd.DataFrame({"Images":images})

Now getting our entire train and test datset into same format for effective use.

### Saving the file

In [ ]:
new_train_data = proc_data_saver(train_data) #uncomment in first run
new_test_data = proc_data_saver(test_data,False) #uncomment in first run
new_valid_data = proc_data_saver(valid_data) #uncomment in first run

<ipython-input-5-732df51364ae>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(data_gen(df.loc[row])[0] / 255, dtype=torch.float64) for row in range(df.shape[0])]
<ipython-input-5-732df51364ae>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = [torch.tensor(data_gen(df.loc[row])[1], dtype=torch.float64) for row in range(df.shape[0])]
<ipython-input-5-732df51364ae>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(data_gen(df.loc[row],False)/ 255, dtype=torch.float64) for row in range(d

In [ ]:
#import pickle #uncomment first time running
#with open("train.pkl","wb") as f:  #uncomment first time
#    pickle.dump(new_train_data,f) #uncomment first time

In [ ]:
#with open("test.pkl","wb") as f:  #uncomment first time
#    (pickle.dump(new_test_data,f))  #uncomment first time

In [ ]:
#with open("valid.pkl","wb") as f:  #uncomment first time
#    (pickle.dump(new_valid_data,f))  #uncomment first time

In [ ]:
import pickle
with open("train.pkl","rb") as f:
    new_train_data = pickle.load(f)
with open("test.pkl","rb") as f:
    new_test_data = pickle.load(f)
with open("valid.pkl","rb") as f:
    new_valid_data = pickle.load(f)

### Conclusion of Part 1

In this markdown, we have learned how to get the data ready for a computer vision project using the MNIST dataset. We have also created a simple function to generate training data and saving it.

## Getting our model Architecture Part 2

In [ ]:
class Model(torch.nn.Module):
    def __init__(self,H,W):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1,out_channels=2,kernel_size=2,padding=1,dtype=torch.float64)
        self.conv2 = torch.nn.Conv2d(in_channels=2,out_channels=4,kernel_size=2,padding=1,dtype=torch.float64)
        self.lin1 =  torch.nn.Linear(in_features=3600,out_features=8*H*W,dtype=torch.float64)
        self.lin5 =  torch.nn.Linear(in_features=8*H*W,out_features=10,dtype=torch.float64)
        self.softmax = torch.nn.Softmax()
    def forward(self,x):
        x = torch.nn.functional.elu(self.conv1(x))
        x = torch.nn.functional.elu(self.conv2(x))
        x = torch.flatten(x)
        x = torch.nn.functional.elu(self.lin1(x))
        x = self.softmax(self.lin5(x))
        return x

In [ ]:
model = Model(28,28)

In [ ]:
model.to('cuda')

Model(
  (conv1): Conv2d(1, 2, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(2, 4, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (lin1): Linear(in_features=3600, out_features=6272, bias=True)
  (lin5): Linear(in_features=6272, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)

## Creating Dataloaders Part 3

### Creating Datasets.
In Pytorch there are two datasets type available
- map-style datasets
- iterable-style datasets.
map-styled datasets are Datasets which inherits the `torch.utils.data.Dataset` and have `__len__`(for length of dataset) and `__getitem__`(to get particular dataitem) defined.  
iterable-styled datasets are Datasets which inherits the `torch.utils.data.IterableDataset` and has `__iter__` (to iterate from one datapoint to another) method defined.
-----------
Here we will use map-styled dataset

In [ ]:
class Data(torch.utils.data.Dataset):
    def __init__(self,df,train=True):
        self.train = train
        if  self.train:
            self.label = df['labels']
            self.images = df['Images']
        else:
            self.images = df['Images']
    def __len__(self):
        return self.images.shape[0]
    def __getitem__(self, index):
        if self.train:
            return self.images[index],self.label[index]
        return self.images[index]

In [ ]:
datasetTrain = Data(new_train_data)
datasetTest = Data(new_test_data,False)
datasetValid = Data(new_valid_data)

Now simply adding the dataset to respective dataloaders.

In [ ]:
# creating train, test dataloader
DataLTrain = torch.utils.data.DataLoader(datasetTrain,batch_size=64,shuffle=True,num_workers=4)
DataLTest = torch.utils.data.DataLoader(datasetTest,batch_size=64,shuffle=False,num_workers=4)
DataLValid = torch.utils.data.DataLoader(datasetValid,batch_size=64,shuffle=True,num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


_______
## Choosing loss function and optimiser (Training) Part 4



----------------

In [ ]:
loss_func = torch.nn.functional.cross_entropy
optimiser = torch.optim.Adam(model.parameters(),lr=0.00001)

In [ ]:
for epoch in range(100):
  for (images_train_batch, label_train_batch),(images_valid_batch, label_valid_batch) in zip(DataLTrain,DataLValid):
    train = 0
    valid = 0
    batch = len(label_train_batch)

    for image, label in zip(images_train_batch,label_train_batch):
        optimiser.zero_grad()
        preds = model(image.reshape(-1,28,28).to('cuda',dtype=torch.float64))
        loss_train = loss_func(preds,label.to('cuda'))
        train += loss_train
        loss_train.backward()
        optimiser.step()
    with torch.inference_mode():
      for image, label in zip(images_valid_batch,label_valid_batch):
          preds = model(image.reshape(-1,28,28).to('cuda',dtype=torch.float64))
          loss_valid = loss_func(preds,label.to('cuda'))
          valid += loss_valid
  print(f"|epoch : {epoch}")
  print(f"|train loss : {train}||valid loss : {valid}")
  train = 0
  valid = 0


## Saving the pytorch model

In [ ]:
torch.save(model.state_dict(),"model.pt")


## Loading pytorch model parameters

In [ ]:
model2 = Model(28,28)
model2.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

## Evaluating on some examples

In [ ]:

model2.eval()
with torch.no_grad():
  pred = model2((data_gen(test_data.loc[5],False)/255).reshape(-1,28,28))
print(torch.round(pred))

(transform(data_gen(test_data.loc[5],False)/255))


tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=torch.float64)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [ ]:
model2.eval()
with torch.no_grad():
  pred = model2((data_gen(test_data.loc[10],False)/255).reshape(-1,28,28))
print(torch.round(pred))

(transform(data_gen(test_data.loc[10],False)/255))

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=torch.float64)
